# Week 3: Improve MNIST with Convolutions

In the videos you looked at how you would improve Fashion MNIST using Convolutions. For this exercise see if you can improve MNIST to 99.5% accuracy or more by adding only a single convolutional layer and a single MaxPooling 2D layer to the model from the  assignment of the previous week. 

You should stop training once the accuracy goes above this amount. It should happen in less than 10 epochs, so it's ok to hard code the number of epochs for training, but your training must end once it hits the above metric. If it doesn't, then you'll need to redesign your callback.

When 99.5% accuracy has been hit, you should print out the string "Reached 99.5% accuracy so cancelling training!"


In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [2]:
# load the data
current_dir = os.getcwd()

data_path = os.path.join(current_dir, 'data/mnist.npz')

(training_images, training_labels), _ = tf.keras.datasets.mnist.load_data(path=data_path)

print(training_images.shape)

(60000, 28, 28)


In [3]:
def reshape_and_normalize(images):
    images = np.reshape(images, (60000, 28, 28, 1))
    
    images = images/255.0
    
    return images

In [4]:
# check the image after reshape and normalize

#load again in case you run the cell multiple times
(training_images, _), _ = tf.keras.datasets.mnist.load_data(path=data_path)

training_images = reshape_and_normalize(training_images)

print(f'Maximum pixels after normalizing images {np.max(training_images)}')
print(f'THe size of training image after reshaping {training_images.shape}')
print(f'The size of each image after reshaping {training_images[0].shape}')

Maximum pixels after normalizing images 1.0
THe size of training image after reshaping (60000, 28, 28, 1)
The size of each image after reshaping (28, 28, 1)


In [5]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('accuracy') > 0.995):
            print("\nReached 99.5% accuracy so cancelling training!")
            self.model.stop_training = True

In [6]:
def convolution_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(28, 28, 1)),
        tf.keras.layers.MaxPooling2D((2,2)),
        
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

In [7]:
model = convolution_model()

callbacks = myCallback()

history = model.fit(training_images, training_labels, epochs=10, callbacks=callbacks)

Epoch 1/10
1875/1875 [==============================] - 27s 14ms/step - loss: 0.1559 - accuracy: 0.9536
Epoch 2/10
1875/1875 [==============================] - 26s 14ms/step - loss: 0.0528 - accuracy: 0.9838
Epoch 3/10
1875/1875 [==============================] - 26s 14ms/step - loss: 0.0330 - accuracy: 0.9898
Epoch 4/10
1875/1875 [==============================] - 26s 14ms/step - loss: 0.0215 - accuracy: 0.9931
Epoch 5/10
1875/1875 [==============================] - 34s 18ms/step - loss: 0.0156 - accuracy: 0.9950
Epoch 6/10
1874/1875 [============================>.] - ETA: 0s - loss: 0.0098 - accuracy: 0.9968
Reached 99.5% accuracy so cancelling training!
1875/1875 [==============================] - 47s 25ms/step - loss: 0.0098 - accuracy: 0.9968


In [8]:
print(f'Accuracy reached after training for {len(history.epoch)} epochs')

Accuracy reached after training for 6 epochs
